In [1]:
import api
import ipywidgets as w
from IPython.display import display, clear_output

In [3]:
# Widgets
# ---

# Elección de curso completo (borra el horario anterior)
wcurso = w.Dropdown(
    options = [('-', 0), ('Primero', 1), ('Segundo', 2), ('Tercero', 3), ('Cuarto', 4)],
    value = 0,
    description = 'Curso:',
)
def wcurso_callback(e):
    global horario
    horario = api.horario_curso(e['new'], wcuatri.value, wgrupo.value, 'materias.json')
    update()
wcurso.observe(wcurso_callback, names='value')

# Elección de cuatrimestre (borra el horario anterior)
wcuatri = w.Dropdown(
    options = [('Primero', 1), ('Segundo', 2)],
    value = 1,
    description = 'Cuatri:',
)
def wcuatri_callback(e):
    global horario
    horario = api.horario_curso(wcurso.value, e['new'], wgrupo.value, 'materias.json')
    update()
wcuatri.observe(wcuatri_callback, names='value')

# Elección de grupo
wgrupo = w.BoundedIntText(
    value = 1,
    min = 1,
    max = 5,
    step = 1,
    description = 'Grupo:',
    disabled = False
)
def wgrupo_callback(e):
    global horario
    update()
wgrupo.observe(wgrupo_callback, names='value')

# Mostrar en pantalla
# ---

def to_widget(x):
    out = w.Output()
    with out:
        display(x)
    return out

def update():
    global horario
    clear_output(wait=True)
    display(w.Box([wcurso, wcuatri, wgrupo]))
    display(w.Box([to_widget(api.formato_horario(horario))]))
    
horario = api.horario_curso(1, 1, 1, 'materias.json')
update()

Box(children=(Dropdown(description='Curso:', options=(('-', 0), ('Primero', 1), ('Segundo', 2), ('Tercero', 3)…

Box(children=(Output(),))

In [4]:
f = open('horario.html','w')
f.write(horario.to_html())
f.close()